### Debugging the cameras

In [2]:
import open3d as o3d
import numpy as np
import os
import json

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def create_camera_mesh():
    # Create a pyramid to represent the camera
    vertices = np.array([[0, 0, 0], [-0.1, -0.1, 0.2], [0.1, -0.1, 0.2], [0., 0.1, 0.2]])
    triangles = np.array([[0, 2, 1], [0, 3, 2], [0, 1, 3], [1, 2, 3]])
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    mesh.triangles = o3d.utility.Vector3iVector(triangles)

    # Create a color for each triangle
    colors = np.array([[1, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1]])

    # Assign the colors to the mesh
    mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

    return mesh

def set_camera_pose(mesh, c2w):
    # Set the mesh's pose according to the c2w matrix
    mesh.transform(c2w)

def render_cameras(w2c_matrices, point_cloud):
    # Create a visualizer
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    points = []
    directions = []

    # Create and add a camera mesh for each w2c matrix
    for w2c in w2c_matrices:
        c2w = np.linalg.inv(w2c)  # Convert from world-to-camera to camera-to-world
        mesh = create_camera_mesh()
        set_camera_pose(mesh, c2w)
        vis.add_geometry(mesh)

        camera_position = c2w[:3, 3]
        viewing_direction = c2w[:3, 2]

        points.append(camera_position)
        directions.append(viewing_direction)

        line_length = -6  # You can adjust this value to change the length of the line
        line_points = np.array([camera_position - line_length * viewing_direction, camera_position + line_length * viewing_direction])
        lines = np.array([[0, 1]])
        line_colors = np.array([[1, 0, 0]])
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector(line_points)
        line_set.lines = o3d.utility.Vector2iVector(lines)
        line_set.colors = o3d.utility.Vector3dVector(line_colors)
        vis.add_geometry(line_set)

    points = np.array(points)
    directions = np.array(directions)

    # Create a coordinate frame
    coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])

    # Add the coordinate frame to the visualizer
    vis.add_geometry(coord_frame)

    # Create a point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)

    # Add the point cloud to the visualizer
    vis.add_geometry(pcd)

    # Render the scene
    vis.run()
    vis.destroy_window()

    return points, directions

In [4]:
with open('C:\kedaxiaoqiu/NeurIPS_baseline/Panoptic/data/basketball/train_meta.json', 'r') as f:
    example_meta = json.load(f)
example_w2cs = np.array(example_meta['w2c'])[0]

In [5]:
example_pts = np.load('C:\kedaxiaoqiu/NeurIPS_baseline/static_ckpts/camc_basketball_static.npz')['means3D']

render_cameras(example_w2cs, example_pts)

(array([[ 2.09252 , -3.12421 , -0.101755],
        [-0.472404, -3.39024 ,  1.80043 ],
        [ 2.0571  , -1.51078 ,  1.78279 ],
        [ 1.56329 , -3.39428 ,  1.00426 ],
        [-1.07326 , -1.52569 ,  2.54681 ],
        [-0.685081, -3.39461 , -1.72309 ],
        [ 0.738283, -3.12108 ,  1.97383 ],
        [-0.631486, -1.52684 , -2.68656 ],
        [ 2.62697 , -1.18364 ,  0.711014],
        [-1.40926 , -2.39556 , -2.11016 ],
        [-0.113029, -2.39425 ,  2.54157 ],
        [-1.86064 , -3.39361 ,  0.117074],
        [ 1.50766 , -1.20287 , -2.32646 ],
        [ 1.57309 , -2.40536 , -1.998   ],
        [-1.75714 , -3.12279 , -1.15322 ],
        [ 2.37421 , -2.38918 ,  0.880995],
        [ 1.42427 , -3.39378 , -1.17698 ],
        [-2.01916 , -1.97938 ,  1.77828 ],
        [-2.44306 , -2.39611 ,  0.683858],
        [ 1.05133 , -1.96766 ,  2.45709 ],
        [ 2.33415 , -1.52574 , -1.43071 ],
        [-2.56741 , -1.194   ,  0.981597],
        [-1.71514 , -1.19045 , -2.12953 ],
        [-1

In [17]:
with open('../../data/mbp/train_meta.json', 'r') as f:
    my_meta = json.load(f)
my_w2cs = np.array(my_meta['w2c'])[0]
corrected_w2cs = my_w2cs.copy()
corrected_w2cs[:, 0] = -corrected_w2cs[:, 0]
corrected_w2cs[:, 1] = -corrected_w2cs[:, 1]
corrected_w2cs[:, 2] = -corrected_w2cs[:, 2]

random_pts = (np.random.rand(20000, 3) - 0.5) * 4

render_cameras(np.concatenate([my_w2cs, corrected_w2cs], axis=0), random_pts)

(array([[-1.38624103,  2.46542814, -2.99999985],
        [-0.66967083,  2.7480067 , -2.99999998],
        [ 0.09656564,  2.82677814, -3.00000001],
        [ 0.8556404 ,  2.69590054, -2.99999999],
        [ 1.55125626,  2.36508014, -3.00000006],
        [ 2.55428084,  1.21476252, -2.99999998],
        [ 2.7873003 ,  0.48057913, -3.00000009],
        [ 2.81359818, -0.28924666, -2.99999995],
        [ 2.63122499, -1.03762048, -2.99999995],
        [ 2.25370512, -1.70903861, -2.99999993],
        [ 1.03762053, -2.631225  , -3.00000008],
        [ 0.28924668, -2.81359836, -2.99999995],
        [-0.48057915, -2.7873003 , -2.99999994],
        [-1.21476258, -2.55428103, -2.99999981],
        [-1.85885272, -2.13182231, -3.00000001],
        [-2.69590056, -0.85564036, -2.99999998],
        [-2.82677828, -0.09656564, -3.00000014],
        [-2.74800689,  0.66967089, -2.9999999 ],
        [-2.46542811,  1.38624103, -2.99999981],
        [-1.99999998,  1.99999998, -2.99999994],
        [-1.38624103